<a href="https://colab.research.google.com/github/erickx22x/AI-2022-googlecolab/blob/main/Avalia%C3%A7%C3%A3o_A1teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports necessários

In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
%matplotlib inline

# Obtenção da base de dados

Base de dados sobre benchmarks de placas de vídeo feitas utilizando as ferramentas mais conhecidas.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
boards = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GPU_benchmarks.csv')


In [ ]:
boards.head()

In [ ]:
boards.info()

In [ ]:
boards.describe()

In [ ]:
boards.corr()

# Análise exploratória


exploração dos dados e comparação dos gráficos para definir se é possível fazer predição com essa base

In [ ]:
figura = plt.figure(figsize=(15,15))
sns.heatmap(boards.corr(),annot = True);

In [ ]:
plt.hist(x=boards['price'])

A princípio serão comparadas as colunas TDP e G3Dmark (nota da ferramenta, quanto mais alta melhor)

In [ ]:
sns.jointplot(data = boards, x ='TDP', y = 'G3Dmark')

In [ ]:
sns.jointplot(data=boards, x='TDP', y='powerPerformance', kind="hex")

In [ ]:
sns.pairplot(boards)

# Preparação dos Dados

Drop de valores nulos em cada coluna específica


In [ ]:
boards = boards.dropna(subset=['price'])
boards = boards.dropna(subset=['TDP'])
boards = boards.dropna(subset=['powerPerformance'])

Drop de coluna do tipo string

In [ ]:
boards.drop('gpuName', axis = 1, inplace=True)
boards.drop('category', axis = 1, inplace=True)


In [ ]:
boards.head()

Alteração do dtype da coluna G2Dmark e G3Dmark para melhor análise

In [ ]:
boards['G3Dmark'] = boards['G3Dmark'].astype(float)

In [ ]:
boards['G2Dmark'] = pd.to_numeric(boards['G2Dmark'].str.replace(',', ''))

Separação de x data e y data para treinamento e previsão

In [ ]:
x_data = boards.drop(columns=['G3Dmark'], axis = 1 )


In [ ]:
y_data = boards.iloc[:,0]


Normalização dos dados

In [ ]:
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)


In [ ]:
x_data_treinamento, x_data_teste, y_data_treinamento, y_data_teste = train_test_split(x_data, y_data, test_size = 0.2, random_state = 25)

#Aplicação do modelo de regressão linear proposto

##Treinamento

In [ ]:
regressao = LinearRegression()
regressao.fit(x_data_treinamento, y_data_treinamento)

In [ ]:
regressao.coef_


In [ ]:
coef_df = pd.DataFrame(regressao.coef_, columns=['Coeficiente'])
coef_df

##Previsão de dados de teste

Os ajustes ao modelo foram feitos, agora é o momento da avaliação do desempenho e prever os valores de teste

In [ ]:
regressao.score(x_data_teste, y_data_teste), regressao.score(x_data_treinamento, y_data_treinamento)

In [ ]:
predicao = regressao.predict(x_data_teste)

In [ ]:
sns.scatterplot(x=y_data_teste, y=predicao)

# Avaliação do modelo

##Resultados

Podemos observar que o desempenho do modelo proposto é considerado bom visto que atinge 83% de acurácia

In [ ]:
from sklearn import metrics
print('MAE', metrics.mean_absolute_error(y_data_teste, predicao))
print('MSE', metrics.mean_squared_error(y_data_teste, predicao))
print('RMSE', np.sqrt(metrics.mean_squared_error(y_data_teste, predicao)))
print('R^2 : ', metrics.r2_score(y_data_teste, predicao))

## Resíduos

E por fim o cálculo dos resíduos para confirmação da manipulação e exibição dos dados bem como regressão linear proposta

In [ ]:
residuos = y_data_teste - predicao

In [ ]:
plt.hist(residuos)